## Example: Initialization and Primitives of the Executor

This example shows different ways to initialize the Executor und run various jobs.

In [1]:
from qiskit_aer import Aer
from qiskit.circuit.random import random_circuit
from qiskit.primitives import Sampler, Estimator, BackendSampler, BackendEstimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import Estimator as RuntimeEstimator

from squlearn.util import Executor
from squlearn.util.executor import BaseEstimatorV1, BaseSamplerV1

The following cells shows different ways to initialize of the ``Executor`` class with Qiskit backends or services.

In [2]:
# from a string representing the simple Qiskit Aer simulators:
executor = Executor("statevector_simulator")
executor = Executor("qasm_simulator")
# from a backend following the Qiskit backend standard:
executor = Executor(Aer.get_backend("aer_simulator_statevector"))

In [3]:
# from a backend obtained from the Qiskit IBM runtime service (here, the account has to be set-up previously):
service = QiskitRuntimeService(channel="ibm_quantum")
# Alternative: service = QiskitRuntimeService(channel="ibm_quantum",token="YOUR_TOKEN_HERE")
executor = Executor(service.get_backend("ibm_kyoto"))
# from a session set-up with the Qiskit IBM runtime service:
session = Session(service, backend=service.get_backend("ibm_kyoto"), max_time=28800)
executor = Executor(session)

# from the Qiskit IBM runtime Estimator primitive:
session = Session(service, backend=service.get_backend("ibm_kyoto"), max_time=28800)
estimator = RuntimeEstimator(session=session)
executor = Executor(estimator)

In [3]:
# from a Qiskit simulator primitive:
executor = Executor(Estimator())
executor = Executor(Sampler())
executor = Executor(BackendEstimator(Aer.get_backend("aer_simulator")))
executor = Executor(BackendSampler(Aer.get_backend("aer_simulator")))

C:\Users\mow\AppData\Local\Temp\ipykernel_9240\3565786267.py:2: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  executor = Executor(Estimator())
C:\Users\mow\AppData\Local\Temp\ipykernel_9240\3565786267.py:3: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  executor = Executor(Sampler())
C:\Users\mow\AppData\Local\Temp\ipykernel_9240\3565786267.py:4: DeprecationWarning: The class ``qiskit.p

The cells demonstrates how to set the number of shots utilized in the circuit evaluation.

In [4]:
# Shots can be set by the executor:
print("Current shots as set before:", executor.get_shots())
# Set shots
executor.set_shots(1234)
print("Adjusted shots:", executor.get_shots())
# Reset shots to initial ones:
executor.reset_shots()
print("Reset shots:", executor.get_shots())

Current shots as set before: 1024
Adjusted shots: 1234
Reset shots: 1024


In this cell, we calculate an expectation value using the Estimator primitive, which is accessible through the ``Executor`` class. The executor generates modified Primitives with enhanced functionality, including caching, automatic session management, and logging capabilities. These modified primitives can be seamlessly incorporated into your workflow or other Qiskit routines.

In [5]:
# Generate a random circuit:
circuit = random_circuit(2, 2, seed=0).decompose(reps=1)

# Generate an observable:
observable = SparsePauliOp("ZI")

# Get the Executor Estimator Primitive and call run:
# Depending on the Estimator version, the run call is slightly different
estimator = executor.get_estimator()
if isinstance(estimator, BaseEstimatorV1):
    print(estimator.run(circuit, observable, shots=4321).result())
else:
    print(estimator.run([(circuit, observable)], precision=1 / 4321**0.5).result())

# Get the Executor Sampler Primitive and call run:
# Depending on the Sampler version, the run call is slightly different
sampler = executor.get_sampler()
if isinstance(sampler, BaseSamplerV1):
    print(sampler.run(circuit.measure_all(inplace=False)).result())
else:
    print(sampler.run([(circuit.measure_all(inplace=False),)]).result())

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=0.0), metadata={'target_precision': 0.01521275485262189, 'shots': 4321, 'circuit_metadata': {}})], metadata={'version': 2})
SamplerResult(quasi_dists=[{1: 1.0}], metadata=[{'shots': 1024}])


The executor can also be used to execute ``backend.run()``. However, caching is not yet implemented for this case.

In [6]:
job = executor.qiskit_execute(circuit)
job.result()

Result(backend_name='aer_simulator', backend_version='0.15.1', qobj_id='', job_id='48c57470-07b5-49f3-93b9-f71ba888904e', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(), header=QobjExperimentHeader(creg_sizes=[], global_phase=0.0, memory_slots=0, n_qubits=2, name='circuit-166', qreg_sizes=[['q', 2]], metadata={}), status=DONE, seed_simulator=169399611, metadata={'batched_shots_optimization': False, 'required_memory_mb': 0, 'method': 'stabilizer', 'active_input_qubits': [], 'device': 'CPU', 'remapped_qubits': False, 'num_qubits': 0, 'num_clbits': 0, 'time_taken': 1.77e-05, 'input_qubit_map': [], 'max_memory_mb': 16135, 'measure_sampling': False, 'parallel_shots': 1, 'parallel_state_update': 12, 'runtime_parameter_bind': False, 'num_bind_params': 1}, time_taken=1.77e-05)], date=2024-10-25T12:05:24.765079, status=COMPLETED, header=None, metadata={'omp_enabled': True, 'parallel_experiments': 1, 'max_memory_mb': 16135, 'max_gpu_me

Additionally, the executor maintains a detailed log of background operations. This feature proves especially valuable when optimizing real backends, allowing you to gain insights into the underlying processes and activities.

In [7]:
executor = Executor("qasm_simulator", log_file="example_log.log")
executor.set_shots(1234)
estimator = executor.get_estimator()

# Depending on the Estimator version, the run call is slightly different
if isinstance(estimator, BaseEstimatorV1):
    print(estimator.run(circuit, observable, shots=4321).result())
else:
    print(estimator.run([(circuit, observable)], precision=1 / 4321**0.5).result())

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=0.0), metadata={'target_precision': 0.01521275485262189, 'shots': 4321, 'circuit_metadata': {}})], metadata={'version': 2})


The executor has a cache where it stores and can reuse job results. In this example, we change the number of shots to tell apart the first and second runs of the same job, and both runs are stored in separate caches. However, the third job simply reuses the cached result from its first execution.

In [8]:
executor = Executor(
    BackendSampler(Aer.get_backend("qasm_simulator")),
    log_file="example_log_cache.log",
    caching=True,
    cache_dir="_cache",
)
executor.set_shots(4321)
estimator = executor.get_estimator()
# Depending on the Estimator version, the run call is slightly different
if isinstance(estimator, BaseEstimatorV1):
    print(estimator.run(circuit, observable).result())
else:
    print(estimator.run([(circuit, observable)]).result())

executor.set_shots(1234)
# Depending on the Estimator version, the run call is slightly different
if isinstance(estimator, BaseEstimatorV1):
    print(estimator.run(circuit, observable).result())
else:
    print(estimator.run([(circuit, observable)]).result())
# This one is load from the cached and not executed again
executor.set_shots(4321)
# Depending on the Estimator version, the run call is slightly different
if isinstance(estimator, BaseEstimatorV1):
    print(estimator.run(circuit, observable).result())
else:
    print(estimator.run([(circuit, observable)]).result())

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=0.0), metadata={'target_precision': 0.01521275485262189, 'shots': 4321, 'circuit_metadata': {}})], metadata={'version': 2})
PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=0.0), metadata={'target_precision': 0.01521275485262189, 'shots': 4321, 'circuit_metadata': {}})], metadata={'version': 2})
PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=0.0), metadata={'target_precision': 0.01521275485262189, 'shots': 4321, 'circuit_metadata': {}})], metadata={'version': 2})


C:\Users\mow\AppData\Local\Temp\ipykernel_9240\1334709714.py:2: DeprecationWarning: The class ``qiskit.primitives.backend_sampler.BackendSampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `BackendSampler` class is `BackendSamplerV2`.
  BackendSampler(Aer.get_backend("qasm_simulator")),
